# Bedrock のログを有効化してみよう

Knowledge bases for Amazon Bedrock では次のログが利用可能です。

1. [Model invocation logging](https://docs.aws.amazon.com/bedrock/latest/userguide/model-invocation-logging.html)
2. [Knowledge bases logging](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-bases-logging.html)

今回はそれぞれのログ記録を有効化し、各ログに記録されるデータを見てみましょう。

## Model invocation logging

Model invocation logging は LLM 全般の実行履歴を記録します。次の形式の生成を行なった場合におけるログを記録します。

- Text
- Image
- Embedding

[Using the console](https://docs.aws.amazon.com/bedrock/latest/userguide/model-invocation-logging.html)

### 有効化

ログの有効化は Terraform でサポートされていないため、マネジメントコンソールより有効化作業を行います。Bedrock コンソールより `設定` からモデル呼び出しのログ記録を有効化します。

以下の情報を指定します。設定を保存します。

- ロググループ名：`/aws/bedrock/devio-2024-invoke`
- サービスロール：`devio-2024-bedrock-log-role`

![](../images/05/01.png)